In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
from handler import geozoom_handler
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from attn_model2 import *
from fc_net import *
from helpers import *
from utils2 import *

In [ ]:
# random.sample(range(0, len(image_names)), 5)

In [ ]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 5)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNetBinary(in_channels = 512, h = 7, w = 7, batch_size = 1, resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)



butler = geozoom_handler(attn_model, fc_model, device, criterion, attn_optimizer, fc_optimizer, plot = False, v = True)

In [ ]:
BATCH_SIZE = 1
SPLIT = .90

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [ ]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape, o)

In [ ]:
501 * .70

In [ ]:
butler.train_attn_model(train_dl, val_dl)

In [ ]:
butler.image_sizes

In [ ]:
butler.threshold_weights.keys()

In [ ]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape)

In [ ]:
641 * .70

In [ ]:
263 - 

In [8]:
for i, o in train_dl:
    
    muni_id = i[0].split("/")[4]
    
    plt.imshow(butler.prep_input(i).detach().cpu()[0].permute(1,2,0))
    plt.title(butler.image_sizes[muni_id])
    plt.savefig(f"test{muni_id}.png")
    
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [9]:
# # for i in range(50):
# while handler.threshold_index > 0:

#     for impath, output in train_dl:

#         # Prep the input and pass it to the trainer (this could easily be done in one step eventually if ya want)
#         input = handler.prep_input(impath)
#         handler.train(input, output)
    
#     handler.end_epoch(train_dl, val_dl = None)

In [8]:
handler.image_sizes

{'484002003': (0, 224, 0, 224),
 '484016045': (0, 224, 0, 224),
 '484020489': (0, 224, 0, 224),
 '484020254': (0, 224, 0, 224)}

In [19]:
butler.threshold_weights.keys()

dict_keys([6, 5, 4, 3, 2, 1, 'fc'])

In [10]:
for i, o in val_dl:
    print(i, o)

('../pooling/data/MEX2/484020535/pngs/484020535_2010_1_box484020535_MAY.png',) tensor([89.], dtype=torch.float64)


In [13]:
def run_validation(attn_model, fc_model, input, weights_dict, plot = False):
    
    muni_id = input[0].split("/")[4]
    cur_image = load_inputs(input[0])    
    
    for k in weights_dict.keys():
                
        if k != 'fc':
            
            model = attn_model
            model.load_state_dict(weights_dict[k])
            model.eval()
            IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
            gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda()) 
            cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)
            
            if plot == True:
            
                plot_gradcam(gradcam)
                plt.savefig(f"threshold{k}_muni{muni_id}.png")
                plt.clf()
            
        
        if k == 'fc':
            
            model = fc_model
            model.load_state_dict(weights_dict[k])
            model.eval()
            return model(cur_image.cuda()).item()
            

        
for i, o in train_dl:

    print(round(run_validation(attn_model, fc_model, i, butler.threshold_weights), 2), "     ", o.item())
    

201.76       196.0
201.08       4.0
273.5       606.0
4416.25       6521.0


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

...

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>